In [1]:
import os
from datetime import datetime, timezone
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
import requests_cache
from pathlib import Path

In [ ]:
# Portfolio of stocks interested
MAG7 = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA"] # the portfolio I am interested in is the magnificent 7
MARKET = "^GSPC"  # using S&P 500 Index as market info (e.g. realized "market" volatility)
RF_SERIES = "TB3MS"  # 3-Month Treasury Bill, Secondary Market Rate (%), monthly from FRED, as the risk-free return r_f

# Range of dates interested
START_DATE = "2023-01-01"
END_DATE = "20250908"  # if 'None', it means today

DATA_DIR = Path(os.path.abspath('')).resolve() / "data"
PRICES_RAW = DATA_DIR / "prices" / "raw"
PRICES_DERIVED = DATA_DIR / "prices" / "derived"
MARKET_DIR = DATA_DIR / "market"
RF_DIR = DATA_DIR / "risk_free"

requests_cache.install_cache("yfinance_cache", expire_after=300) # create a cache called 'yfinance_cache.sqlite' for the data fetched for faster re-runs while developing 
# both yfinance (Yahoo) and pandas-datareader benefit


In [ ]:
# Some util functions

def ensure_dirs():
    """
    Create the directories to store the info
    """
    for d in [PRICES_RAW, PRICES_DERIVED, MARKET_DIR, RF_DIR]:
        d.mkdir(parents=True, exist_ok=True)

def _normalize_dates(start, end):
    start = pd.Timestamp(start, tz=timezone.utc).tz_convert(None)
    end = pd.Timestamp.today(tz=timezone.utc).tz_convert(None) if end is None else pd.Timestamp(end, tz=timezone.utc).tz_convert(None)
    return start, end

def fetch_daily_panel(tickers, start, end):
    """
    Returns a MultiIndex-columns DataFrame:
    columns: (field, ticker) for fields in ['Open','High','Low','Close','Adj Close','Volume']
    index: datetime (UTC-naive)
    """
    df = yf.download(
        tickers=tickers,
        start=start,
        end=end,
        auto_adjust=False,   # keep raw + Adj Close; we’ll use Adj Close for returns
        actions=False,
        progress=False,
        group_by="column"
    )
    # yfinance returns columns as a single-level when 1 ticker; unify shape:
    if isinstance(df.columns, pd.MultiIndex):
        pass
    else:
        # single ticker -> promote to MultiIndex
        df = pd.concat({tickers[0]: df}, axis=1)

    # Reorder to (field, ticker) for easier selection later
    df = df.swaplevel(axis=1).sort_index(axis=1)
    # Remove timezone if present
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(None)
    return df

def compute_daily_returns(prices_panel):
    """
    Compute percentage returns from Adj Close for each ticker.
    Returns a wide DataFrame with tickers as columns.
    """
    adj = prices_panel.xs('Adj Close', axis=1, level='Price').copy()
    rets = adj.pct_change().dropna(how="all")
    return rets

def save_df(df, path_no_ext):
    """
    Save both parquet and csv (UTF-8).
    """
    df.to_parquet(f"{path_no_ext}.parquet")
    df.to_csv(f"{path_no_ext}.csv", index=True)

def fetch_market_series_and_realized_vol(start, end):
    """
    Fetch ^GSPC daily, compute within-month realized variance & volatility:
        σ2_t = Var_{daily in month t}(r_d)
        σ_t  = sqrt(σ2_t)
    Also compute inverse measures: inv_sigma_t = 1/σ_t and inv_var_t = 1/σ2_t
    Returns a month-end indexed DataFrame.
    """
    mkt = yf.download(MARKET, start=start, end=end, auto_adjust=True, progress=False)
    if mkt.empty:
        raise RuntimeError("Failed to fetch market series.")
    mkt.index = pd.to_datetime(mkt.index, utc=True).tz_convert(None)
    price = (mkt.xs('Close', axis=1, level='Price') if isinstance(mkt.columns, pd.MultiIndex) else mkt['Close']).squeeze()
    
    # Within-month realized variance (sample variance of daily returns)
    # Use ddof=1 to match sample variance convention
    ret = price.pct_change()
    rv = ret.groupby(pd.Grouper(freq="ME")).agg(lambda s: s.dropna().var(ddof=1))
    
    sigma2 = rv.rename("sigma2")
    sigma = sigma2.pow(0.5).rename("sigma")
    inv_sigma = (1.0 / sigma).replace([pd.NA, pd.NaT, float("inf")], pd.NA).rename("inv_sigma")
    inv_sigma2 = (1.0 / sigma2).replace([pd.NA, pd.NaT, float("inf")], pd.NA).rename("inv_sigma2")
    out = pd.concat([sigma2, sigma, inv_sigma, inv_sigma2], axis=1).dropna()
    return out

def fetch_rf_monthly(start, end):
    """
    Fetch TB3MS (%) monthly from FRED and convert to monthly return:
        rf_month = (1 + rate/100) ** (1/12) - 1
    """
    # FRED returns month-end indexed series already
    rf = pdr.DataReader(RF_SERIES, "fred", start, end)
    rf.index = pd.to_datetime(rf.index, utc=True).tz_convert(None)
    rf = rf.rename(columns={RF_SERIES: "TB3MS_pct"})
    rf["rf_month"] = (1.0 + rf["TB3MS_pct"] / 100.0) ** (1.0 / 12.0) - 1.0
    # Keep only month-end, drop NaNs
    rf = rf.dropna()
    return rf


In [41]:
ensure_dirs() # create the directories needed
start, end = _normalize_dates(START_DATE, END_DATE)

# 1) Prices: MAG7 + save raw panel + daily returns
tickers = MAG7.copy()
prices_panel = fetch_daily_panel(tickers, start, end)
save_df(prices_panel, PRICES_RAW / "daily_prices")

daily_rets = compute_daily_returns(prices_panel)
save_df(daily_rets, PRICES_DERIVED / "daily_returns")

# 2) Market: realized monthly vol/var + inverses
market_vol = fetch_market_series_and_realized_vol(start, end)
save_df(market_vol, MARKET_DIR / "market_vol_monthly")

# 3) Risk-free: monthly return
rf = fetch_rf_monthly(start, end)
save_df(rf, RF_DIR / "rf_monthly")

# 4) Quick sanity print
print("\nSaved files:")
print(f"- {Path(*PRICES_RAW.parts[PRICES_RAW.parts.index('data'):])/'daily_prices.parquet'} & .csv  (panel OHLCV)")
print(f"- {Path(*PRICES_DERIVED.parts[PRICES_DERIVED.parts.index('data'):])/'daily_returns.parquet'} & .csv  (AdjClose returns)")
print(f"- {Path(*MARKET_DIR.parts[MARKET_DIR.parts.index('data'):])/'market_vol_monthly.parquet'} & .csv  (σ_t, σ2_t, 1/σ_t, 1/σ2_t)")
print(f"- {Path(*RF_DIR.parts[RF_DIR.parts.index('data'):])/'rf_monthly.parquet'} & .csv  (monthly rf returns)")
print("\nShapes:")
print("  daily_prices   :", prices_panel.shape)
print("  daily_returns  :", daily_rets.shape)
print("  market_vol_m   :", market_vol.shape)
print("  rf_monthly     :", rf.shape)


Saved files:
- data\prices\raw\daily_prices.parquet & .csv  (panel OHLCV)
- data\prices\derived\daily_returns.parquet & .csv  (AdjClose returns)
- data\market\market_vol_monthly.parquet & .csv  (σ_t, σ2_t, 1/σ_t, 1/σ2_t)
- data\risk_free\rf_monthly.parquet & .csv  (monthly rf returns)

Shapes:
  daily_prices   : (671, 42)
  daily_returns  : (670, 7)
  market_vol_m   : (33, 4)
  rf_monthly     : (32, 2)
